In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
url_train = 'https://raw.githubusercontent.com/sebischair/Medical-Abstracts-TC-Corpus/main/medical_tc_train.csv'
df_train = pd.read_csv(url_train,index_col=0,parse_dates=[0],on_bad_lines='skip')
url_test = 'https://raw.githubusercontent.com/sebischair/Medical-Abstracts-TC-Corpus/main/medical_tc_test.csv'
df_test = pd.read_csv(url_test,index_col=0,parse_dates=[0],on_bad_lines='skip')


C:\Users\blomfam\AppData\Local\Temp\ipykernel_20276\2772293433.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_train = pd.read_csv(url_train,index_col=0,parse_dates=[0],on_bad_lines='skip')
C:\Users\blomfam\AppData\Local\Temp\ipykernel_20276\2772293433.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_test = pd.read_csv(url_test,index_col=0,parse_dates=[0],on_bad_lines='skip')


In [3]:
pred = "1 0 0 1 0 1 4 0 3 4 2 3 1 3 3 0 2 0 2 2 3 4 4 4 3 2 2 1 3 0 1 2 3 2 2 3 0 3 2 4 3 3 4 4 2 1 0 0 2 0 1 0 1 3 1 3 3 3 4 0 3 0 3 3 2 4 2 1 2 3 3 1 4 4 3 0 0 0 3 0 1 3 2 3 4 3 3 3 2 2 3 2 4 0 0 2 0 0 1 3 2 1 4 4 2 0 0 0 1 3 4 2 3 3 2 4 3 0 4 3 2 0 2 3 0 2 2 0 2 0 3 2 3 3 2 3 4 0 2 0 0 4 3 0 1 0 0 4 3 0 0 1 0 4 0 4 3 2 0 1 2 0 3 3 2 3 0 3 3 4 2 3 3 0 1 1 2 3 4 3 0 3 3 1 4 4 4 3 3 0 4 4 3 3 0 0 0 2 2 2 1 1 0 3 0 1 3 4 1 0 4 0 4 0 3 1 1 1 1 3 3 2 4 0 2 1 0 2 2 3 3 4 2 4 3 3 4 3 1 3 0 0 0 4 0 2 1 1 4 0 0 4 0 2 3 0 4 0 0 0 1 1 3 4 0 4 1 3 4 4 4 1 2 4 0 4 4 2 0 2 2 1 4 3 3 0 1 3 3 3 2 0 0 0 2 0 2 3 0 4 3 4 4 4 1 0 3 2 3 2 1 4 4 2 4 1 2 4 4 1 0 2 1 3 0 3 2 1 4 3 4 0 4 0 4 0 4 3 3 0 3 4 0 0 2 1 1 1 1 3 2 1 0 1 3 0 4 2 0 0 4 4 3 0 1 2 4 0 2 2 3 1 0 2 3 3 0 1 0 0 0 2 3 0 0 4 4 3 2 0 4 4 2 4 0 2 2 2 4 2 0 4 3 4 3 4 0 3 2 0 3 2 3 3 3 3 3 3 4 2 4 0 2 0 0 3 1 3 2 3 0 3 2 0 3 4 3 3 3 3 2 4 3 0 4 3 0 1 2 0 3 2 0 1 4 4 0 0 3 3 0 0 0 0 4 0 3 4 3 4 0 2 1 3 4 1 2 0 0 4 4 3 2 0 4 0 0 3 2 3 1 3 2 2 0 4 3 2 0 3 4 2 0 2 0 1 0 2 4 3 0 0 4 0 1 0 1 4 2 2 2 3 0 4 0 3 0 1 0 1 0 0 0 0 3 1 4 3 0 0 2 4 3 0 3 3 3 0 3 3 3 3 1 4 4 1 3 4 3 0 4 0 3 2 0 0 4 0 3 2 1 0 4 3 1 0 2 3 2 4 4 0 0 2 3 0 0 4 0 3 0 4 4 0 0 2 0 4 1 3 3 1 0 1 0 3 2 3 4 3 3 3 2 4 0 0 2 4 0 4 4 1 4 1 0 4 4 0 4 1 3 4 3 3 3 0 4 4 3 2 4 4 1 4 4 4 3 3 3 1 3 0 2 1 2 0 3 1 4 2 3 4 0 0 3 0 3 0 4 0 3 3 0 3 3 3 3 3 3 0 4 0 3 2 4 3 4 0 4 3 1 2 4 1 4 3 4 2 2 4 4 4 4 1 3 4 1 0 1 3 0 1 0 0 0 2 1 2 0 1 3 4 3 4 3 2 3 4 4 0 4 0 0 0 2 1 2 3 0 2 0 0 4 3 3 3 3 4 3 3 1 4 2 1 3 3 0 1 2 0 3 4 3 4 3 0 1 1 1 0 3 2 3 4 1 1 4 1 3 0 4 2 4 0 1 0 0 0 2 1 0 2 3 4 0 1 3 0 2 0 1 3 2 0 3 3 0 3 2 4 0 1 0 3 4 3 1 3 3 4 3 4 0 0 4 0 0 0 0 2 0 4 1 3 2 0 0 3 4 1 2 1 2 0 2 2 1 0 3 2 3 2 4 2 0 0 3 3 0 3 0 0 3 4 0 3 3 4 2 0 2 3 0 1 2 2 4 3 3 3 3 3 1 3 4 1 4 2 3 4 3 0 3 4 0 3 3 2 0 4 0 3 3 2 1 1 0 0 2 3 3 0 1 4 0 0 4 4 0 0 0 1 3 3 3 4 0 0 3 0 0 1 3 0 0 2 3 4 4 0 1 1 1 0 3 2 3 4 0 3 0 4 1 0 3 4 2 4 3 4 0 0 1 0 0 4 4 4 0 2 0 2 1 1 4 1 4 0 4 0 4 3 3 1 4 0 3 0 3 0 1 4 4 4 0 3 1 1 0 0 3 0 0 3 4 0 4 3 2 3 1 4 1 0 2 2 0 3 4 1 3 4 0 0 4 0 3 4 3 3 0 0 2 1 3 0 4 1 3 0 3 0 2 1 3 2 3 0 4 4 3 0 1 0 3 2 3 2 1 3 3 0 0 0 3 0 0 3 3 3 2 0 3 4 3 3 3 3 2 2 0 4 2 0 4 3 4 0 4 1 2 0 3 0 3 3 1 0 0 2 1 0 3 0 0 3 0 1 3 1 1 4 4 3 2 3 2 1 0 0 0 0 1 0 3 2 2 0 2 4 3 3 0 2 2 4 4 3 0 1 4 2 4 2 1 4 4 2 3 3 2 3 0 3 0 3 3 3 3 2 0 4 1 0 3 3 3 0 0 4 1 2 0 0 0 0 0 3 4 1 3 1 1 1 4 0 1 0 4 4 3 4 2 0 1 3 3 2 0 3 2 1 0 1 2 3 0 3 3 2 0 0 0 4 4 0 3 3 4 0 4 0 1 2 3 0 3 0 3 4 3 4 4 4 1 0 4 3 0 4 0 0 1 2 1 2 3 1 1 1 2 3 0 4 4 3 4 4 2 3 3 3 4 0 4 4 4 2 1 0 3 0 4 3 3 4 3 3 4 4 1 2 0 4 3 4 4 0 3 0 4 0 3 0 0 3 4 1 4 1 3 1 1 3 4 3 1 4 3 0 3 2 4 2 4 3 0 3 0 4 4 3 3 0 4 4 3 3 2 1 4 4 3 3 3 3 2 3 4 1 0 4 0 4 3 2 4 4 0 3 3 0 3 4 0 2 3 3 4 3 2 3 2 2 4 3 3 2 2 3 4 4 1 0 2 4 3 0 3 3 0 2 2 3 3 1 2 3 0 0 4 4 3 3 3 0 1 3 4 1 1 3 2 1 0 4 0 4 4 4 0 1 0 3 4 4 3 3 1 3 3 0 3 0 1 0 1 1 4 0 0 2 4 2 3 3 1 0 2 4 4 3 1 1"
len(pred)

2887

In [4]:
true = "1 0 0 1 0 1 2 0 3 4 2 3 1 3 3 0 2 0 2 2 3 4 4 4 4 2 4 4 4 0 1 2 3 2 2 4 0 3 2 3 3 4 4 4 3 1 0 0 4 0 1 1 1 3 1 3 3 2 4 4 3 0 3 3 2 3 2 1 4 3 3 4 4 4 4 0 0 0 3 0 1 4 4 3 3 3 3 3 2 2 4 0 2 0 4 4 0 0 4 3 2 1 0 0 2 0 1 0 4 4 2 2 3 3 2 0 4 3 4 3 2 0 0 3 0 2 4 0 0 0 3 2 4 3 2 4 2 0 4 4 0 4 3 0 1 0 4 4 3 0 2 4 0 4 4 4 3 2 0 4 2 0 3 4 2 4 0 0 3 4 4 3 4 4 1 1 3 3 4 3 0 3 3 4 4 4 4 4 3 0 4 4 3 3 0 0 0 2 2 4 1 1 0 4 0 1 4 4 4 4 4 4 4 0 3 1 4 1 3 3 3 2 4 0 4 4 4 2 0 3 4 4 2 4 3 3 4 3 2 3 0 0 0 4 0 2 4 1 2 0 4 0 4 2 3 0 2 4 0 0 4 1 4 4 2 4 4 3 4 4 4 4 2 1 4 4 1 4 0 2 2 4 4 3 3 4 4 4 3 3 2 0 0 4 4 0 3 4 0 4 3 4 4 2 4 1 3 2 4 2 4 4 4 2 2 4 4 4 3 4 0 4 1 4 4 3 2 0 1 3 4 1 4 4 4 4 0 3 3 4 3 1 4 4 2 1 4 3 4 4 2 1 0 3 3 0 4 2 0 0 1 4 4 0 1 2 2 0 2 2 4 4 0 4 3 3 4 1 4 0 0 2 4 0 4 2 3 3 0 0 4 3 2 3 0 4 2 2 1 2 0 0 4 3 4 0 0 4 3 4 3 3 3 3 4 4 3 3 0 2 2 0 2 0 0 3 4 3 4 3 0 3 2 0 3 4 3 2 3 3 2 4 3 4 4 4 0 1 2 0 4 2 0 1 4 4 0 0 3 3 0 0 0 0 4 0 4 4 3 4 0 2 4 4 4 0 2 4 0 4 4 3 2 0 3 0 0 3 4 3 4 3 2 2 0 2 3 2 0 4 2 4 0 3 4 1 0 4 4 3 0 0 4 4 1 4 2 1 2 2 4 2 0 4 4 3 0 1 1 1 0 0 3 1 3 1 4 3 0 4 2 3 3 0 3 4 3 0 3 3 4 4 2 4 0 1 3 4 1 1 3 0 4 2 2 0 4 0 3 2 1 0 4 3 2 4 2 4 2 4 3 0 1 2 4 0 0 4 0 2 0 4 4 0 0 2 0 1 2 3 3 0 0 4 0 4 4 4 1 4 3 3 4 4 2 0 4 0 0 4 3 1 4 0 0 4 4 1 0 4 3 0 3 3 3 0 1 4 4 2 4 4 1 1 4 4 4 4 1 4 3 0 0 1 2 0 4 1 4 2 3 4 0 0 3 1 4 0 2 0 3 3 1 3 4 3 3 3 4 4 1 0 3 2 0 3 4 0 4 2 1 3 4 1 4 3 4 2 2 4 4 4 4 1 3 4 1 4 1 4 0 1 0 0 4 2 4 2 4 1 4 4 2 0 3 2 0 4 4 0 4 0 4 4 4 4 2 3 0 3 0 4 4 3 3 3 3 0 3 3 4 4 2 4 3 4 0 1 2 0 3 4 0 4 4 0 4 1 1 0 4 2 4 4 4 1 1 1 3 0 4 4 4 4 1 0 4 1 2 1 0 2 3 3 0 1 3 0 2 0 1 4 2 0 3 4 0 3 2 3 4 0 0 3 1 3 1 3 3 1 3 4 0 0 2 0 4 0 4 2 0 4 4 3 2 0 0 4 2 4 4 1 4 0 0 2 1 0 3 2 4 4 4 4 0 4 3 3 0 3 0 0 3 3 2 4 4 4 4 0 2 4 0 1 2 2 4 3 3 3 0 3 1 3 4 1 4 2 4 4 3 0 3 4 4 3 3 4 0 2 4 4 4 2 1 1 0 0 2 3 4 0 1 4 2 0 4 4 0 0 4 1 3 4 3 4 0 0 3 0 4 1 2 4 0 2 3 2 2 0 4 1 1 0 3 2 3 4 4 3 0 4 4 2 3 4 2 4 3 2 0 0 1 0 4 2 4 4 4 2 0 2 4 1 4 1 4 0 4 0 4 3 3 1 2 0 4 0 4 0 4 4 3 4 0 3 4 1 3 0 3 0 0 3 4 0 4 3 4 4 3 1 1 0 2 2 0 3 2 1 3 0 0 1 4 0 3 0 3 4 0 4 2 4 3 1 1 4 3 0 3 4 2 1 4 2 2 2 4 4 3 0 1 0 4 2 3 2 4 4 4 0 0 0 3 0 0 3 3 3 4 0 3 4 3 3 4 3 3 2 4 4 2 0 4 3 4 0 4 0 2 0 3 4 4 3 4 0 0 2 1 0 3 0 2 0 4 4 3 4 1 4 2 4 4 3 2 4 0 0 0 0 4 0 3 2 2 0 4 4 3 4 0 2 2 0 2 3 0 0 4 2 4 2 4 4 1 4 3 3 4 3 0 4 0 3 4 3 1 2 3 1 2 0 3 3 3 0 4 0 1 4 0 2 2 4 0 3 4 4 3 4 0 1 4 0 1 0 1 4 3 4 0 2 1 2 4 2 0 2 2 1 0 1 4 3 0 3 3 2 0 0 4 4 2 0 3 4 1 0 3 3 1 2 3 0 4 0 4 4 4 0 4 4 4 0 4 3 4 4 4 4 4 4 4 4 4 1 4 1 2 3 0 3 0 3 3 4 2 4 3 1 4 4 4 4 3 2 3 4 4 1 3 3 3 3 3 4 0 4 4 2 0 4 4 4 0 1 4 4 1 4 4 4 0 4 4 1 4 1 4 4 1 4 2 4 1 4 3 4 3 4 3 4 1 3 0 4 4 4 4 3 3 0 4 4 3 3 4 1 0 3 4 1 4 3 2 3 2 4 2 4 4 4 3 2 3 4 1 3 3 0 3 4 2 2 3 3 1 3 0 0 2 4 4 3 3 3 4 3 3 4 1 0 4 4 3 3 3 3 0 2 3 3 4 4 2 3 0 0 4 4 3 3 3 0 1 3 4 4 1 4 2 3 0 4 0 0 4 3 0 4 0 4 4 2 3 3 0 4 3 0 3 0 1 0 0 1 4 0 0 0 4 0 3 3 4 4 2 2 4 3 0 1"


In [15]:
import random
baseline_preds = []
for i in range(10):
    base = []
    for n in range(len(true)):
        base.append(str(random.randint(0,4)))
    baseline_preds.append(base)


In [9]:
len(baseline_preds[0])
true = [x for x in true if x != " "]


In [17]:
from sklearn.metrics import make_scorer, accuracy_score, f1_score
accuracys = []
f1s = []
for pred in baseline_preds:
    accuracys.append(accuracy_score(true, pred))
    f1s.append(f1_score(true, pred, average='weighted'))

In [21]:
avg_acc = np.mean(accuracys)
std_acc = np.std(accuracys)
avg_f1 = np.mean(f1s)
std_f1 = np.std(f1s)

In [22]:
print(avg_acc, std_acc, avg_f1, std_f1)

0.20235457063711912 0.006011782386207952 0.20936115881727407 0.006337097237670311
